In [130]:
# import libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

from sklearn.linear_model import LogisticRegression

In [131]:
train = pd.read_csv('C:/Users/austi/OneDrive/Desktop/Stuff/Academic fun/School/TTS/Titanic work/titanic/train.csv')
test = pd.read_csv('C:/Users/austi/OneDrive/Desktop/Stuff/Academic fun/School/TTS/Titanic work/titanic/test.csv')

In [132]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [133]:
train.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [134]:
#Look for nulls
#train.info()

In [135]:
#See where there are missing values in the training set
train.isnull().sum().sort_values(ascending = False)

Cabin          687
Age            177
Embarked         2
PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Fare             0
dtype: int64

In [136]:
#Find missing values in the testing set
test.isnull().sum().sort_values(ascending=False)

Cabin          327
Age             86
Fare             1
PassengerId      0
Pclass           0
Name             0
Sex              0
SibSp            0
Parch            0
Ticket           0
Embarked         0
dtype: int64

In [137]:
#finding Relationships to survival(Pclass, Sex, and Age)

#Pclass
train[["Pclass", "Survived"]].groupby(["Pclass"], as_index=False).mean()



,Pclass,Survived
0,1,0.629630
1,2,0.472826
2,3,0.242363


In [138]:
#Sex
train[["Sex", "Survived"]].groupby(["Sex"], as_index=False).mean()

,Sex,Survived
0,female,0.742038
1,male,0.188908


In [139]:
#Age

In [140]:
#Combining testing and training data

train_test = [train, test]

In [141]:
# Changing sex to numbers for computation purposes later
for dataset in train_test:
    dataset["Sex"] = dataset["Sex"].map({'female': 1, 'male': 0}).astype(int)

In [142]:
# Note how sex is either 0 or 1 in the output
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",0,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",1,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",0,35.0,0,0,373450,8.0500,NaN,S


In [143]:
#Adding random ages for null age

for dataset in train_test:
    #basic data collection
    average_age = dataset["Age"].mean()
    std_age = dataset["Age"].std()
    null_count = dataset["Age"].isnull().count()
    
    #calculation
    null_random_fill = np.random.randint(average_age - std_age, average_age + std_age, size=null_count)
    # Data fill in
    dataset["Age"][np.isnan(dataset["Age"])] = null_random_fill
    dataset["Age"] = dataset["Age"].astype(int)

#Establishing age bins for grouping
train["AgeBin"] = pd.cut(train["Age"], 5)

print(train[["AgeBin", "Survived"]].groupby(["AgeBin"], as_index=False).mean())

          AgeBin  Survived
0  (-0.08, 16.0]  0.513043
1   (16.0, 32.0]  0.353211
2   (32.0, 48.0]  0.376923
3   (48.0, 64.0]  0.434783
4   (64.0, 80.0]  0.090909


<ipython-input-143-d02af669ae96>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset["Age"][np.isnan(dataset["Age"])] = null_random_fill


In [144]:
#Establishing age bins

for dataset in train_test:
    dataset.loc[ dataset["Age"] <= 13, "Age"] = 0
    dataset.loc[ (dataset["Age"] > 13) & (dataset["Age"] <= 27), "Age"] = 1
    dataset.loc[ (dataset["Age"] > 27) & (dataset["Age"] <= 40), "Age"] = 2
    dataset.loc[ (dataset["Age"] > 40) & (dataset["Age"] <= 53), "Age"] = 3
    dataset.loc[ (dataset["Age"] > 53) & (dataset["Age"] <= 67), "Age"] = 4
    dataset.loc[ dataset["Age"] > 67, "Age"] = 5

In [145]:
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,AgeBin
0,1,0,3,"Braund, Mr. Owen Harris",0,1,1,0,A/5 21171,7.2500,NaN,S,"(16.0, 32.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",1,2,1,0,PC 17599,71.2833,C85,C,"(32.0, 48.0]"
2,3,1,3,"Heikkinen, Miss. Laina",1,1,0,0,STON/O2. 3101282,7.9250,NaN,S,"(16.0, 32.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",1,2,1,0,113803,53.1000,C123,S,"(32.0, 48.0]"
4,5,0,3,"Allen, Mr. William Henry",0,2,0,0,373450,8.0500,NaN,S,"(32.0, 48.0]"


In [146]:
#Eliminating features for predictions
dropped_feat = ["Name", "SibSp", "Parch", "Ticket", "Fare", "Cabin", "Embarked"]
train_dropped = train.drop(dropped_feat, axis=1)
test_dropped = test.drop(dropped_feat, axis=1)

train_dropped = train_dropped.drop(["PassengerId", "AgeBin"],axis=1)



In [152]:
#Establishing data for predictions

x_training = train_dropped.drop("Survived", axis=1)
y_training = train_dropped["Survived"]
x_testing = test_dropped.drop ("PassengerId", axis=1).copy()

x_train.shape, y_train.shape, x_test.shape

((891, 3), (891,), (418, 3))

In [155]:
eq = LogisticRegression()
eq.fit(x_training,y_training)
logistic_reg = eq.predict(x_testing)
final_log_reg = round( eq.score(x_training, y_training) * 100, 2)
print(str(acc_log_reg) + " percent")

79.01 percent


In [158]:
Data_submission_format = pd.DataFrame({"PassengerId": test["PassengerId"], "Survived": logistic_reg})

In [159]:
Data_submission_format

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
